# Datos climáticos diarios

Se generan las series climáticas diarias de las estaciones principales de la AEMET. Se pueden descargar todas las disponibles o por provincias. La duración de la serie también es definible

In [1]:
import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import pandas as pd
import numpy as np
import datetime
import time

In [17]:
# Ruta en la que está guardada la API key
ruta_API = "F:/Series/AEMET/apikey_AEMET.txt"
# Ruta donde se guardarán las series
ruta_series = 'F:/Series/AEMET/Clima/Diario/'

# Definir el inicio y fin de la serie, y la duración de cada periodo
start_date = datetime.datetime(1950, 1, 1, 0, 0, 0)
final_date = datetime.datetime(2017, 11, 30, 23, 59, 59)

# Definir las estaciones: 'todas' o la lista de con los códigos
estaciones = ['5270', '5270B', '5514', '6205X', '6277B', '6293X', '6297', '6302A', '6325O']#, '5047E', '5051X', '5530E', '5582A', '6268X']

#### Funciones

In [3]:
def dms2dd(degrees, minutes, seconds, direction):
    """It converts a latitude or longitude value given in degrees, minutes, seconds and direction to decimal degrees
    """
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(3600);
    if direction == 'W' or direction == 'S':
        dd *= -1
    return dd

## 1. Estaciones

In [4]:
# Carga la api key 
api_key = open(ruta_API).read().rstrip()
querystring = {"api_key": api_key}

# Obtenemos información de todas las estaciones disponibles
url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones"

In [5]:
# Realizamos la request
r = requests.get(url, params=querystring, verify=False)

In [6]:
# Obtenemos el link del que descargaremos los datos
data_url = r.json()['datos']
r_data = requests.get(data_url, params=querystring, verify=False)

In [7]:
# Vemos el contenido
stations = r_data.json()

In [8]:
# Crea un data frame con las estaciones meteorológicas y sus características
n = len(stations)

columnas = []
for key in stations[0].keys():
    columnas.append(key)

stn_AEMET = pd.DataFrame(index=np.arange(n), columns=columnas)
stn_AEMET = stn_AEMET.loc[:, ['indicativo', 'nombre', 'provincia', 'indsinop',
                              'latitud', 'longitud', 'altitud']]

i = 0
for stn in stations:
    stn_AEMET.indicativo[i] = stn['indicativo']
    stn_AEMET.nombre[i] = stn['nombre']
    stn_AEMET.provincia[i] = stn['provincia']
    stn_AEMET.indsinop[i] = stn['indsinop']
    stn_AEMET.latitud[i] = dms2dd(stn['latitud'][:2], stn['latitud'][2:4], stn['latitud'][4:6], stn['latitud'][6])
    stn_AEMET.longitud[i] = dms2dd(stn['longitud'][:2], stn['longitud'][2:4], stn['longitud'][4:6], stn['longitud'][6])
    stn_AEMET.altitud[i] = int(stn['altitud'])
    i += 1
stn_AEMET.sort_values('indicativo', inplace=True)
stn_AEMET.set_index('indicativo', drop=True, inplace=True)

print('Nº de estaciones disponibles: ', stn_AEMET.shape[0])

Nº de estaciones disponibles:  291


In [9]:
stn_AEMET.to_csv(ruta_series + 'estaciones_AEMET.csv', sep=',', na_rep='-100', header=True, index=True)

## 2. Extraer datos diarios

### 2.1 Funciones

In [11]:
def generate_dates(start_date, final_date, step):
    """It generates a list of dates beginning in the 'start_date', ending in the 'final_date' and every 'step' number of 
    dates"""
    
    chunks = [start_date]

    next_date = start_date + step
    while next_date < final_date:
        chunks.append(next_date)
        next_date += step

    chunks.append(final_date)

    return chunks

In [12]:
def extract_data(station, periodos):

    raw_data = []

    for ii in range(1, len(periodos)):
        print()
        print(station,
              " - ",
              periodos[ii-1].strftime('%Y-%m-%dT%H:%M:%SUTC'),
              " - ",
              periodos[ii].strftime('%Y-%m-%dT%H:%M:%SUTC'))

        url = ("https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/"
               "fechaini/{start}/fechafin/{end}/estacion/{station}".format(
                   start=periodos[ii-1].strftime('%Y-%m-%dT%H:%M:%SUTC'),
                    end=periodos[ii].strftime('%Y-%m-%dT%H:%M:%SUTC'),
                    station=station)
              )

        iterate = True

        while iterate:

            r = requests.get(url, params=querystring, verify=False)

            # Si no me deja hacer la conexión, la repito  
            iterate = (r.status_code == requests.codes.too_many_requests)

            print(r.json())

            # Chequeo si la petición ha ido bien    
            if r.status_code == requests.codes.ok:
                # Hago la petición para obtener los datos
                data_url = r.json()['datos']
                r_data = requests.get(data_url, params=querystring, verify=False)

                # INCONSISTENCIA DE LA API:
                # Cuando no encuentra datos en el rango seleccionado, la API devuelve
                # que el status code es 200 (todo ok) y devuelve un json con el error
                # cuando encuentra, no hay atributo estado            
                try:
                    estado = r_data.json()['estado']
                except:
                    estado = 200


                # Si ha ido bien guardo los datos
                if estado == requests.codes.ok:
                    #print(r_data.json())
                    raw_data.extend(r_data.json())
                else:
                    print(r_data.json()['descripcion'])
            else:
                print(r.json()['descripcion'])

            time.sleep(60/45)
            
    return raw_data

In [35]:
def parse_data(raw_data):
    """Convert from string to floating point number those columns in the data that represent quantitative variables"""
    data = []
    for d in raw_data:
        d = dict(d)  # Exto copia el parámetro
        for param in ['prec', 'presMax', 'presMin', 'racha', 'sol', 'tmax', 'tmed', 'tmin', 'velmedia', 'altitud', 'dir']:
            try:
                d[param] = float(d[param].replace(',', '.'))
            except:
                d[param] = None
        data.append(d)
    
    return data

In [13]:
def serie_df(data):
    """Create a data frame with the series of daily records of a station given as a list of dictionaries"""
    # Crea un data frame con los datos diarios de una estación
    n = len(data)

    columnas = []
    for key in data[0].keys():
        columnas.append(key)

    serie = pd.DataFrame(index=np.arange(n), columns=columnas)
    serie = serie.loc[:, ['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'prec', 'tmed', 'tmin', 'horatmin', 'tmax', 
                          'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin',
                          'horaPresMin']]

    for i in np.arange(len(data)):
        keys = list(data[i].keys())
        for key in keys:
            serie[key][i] = data[i][key]
    
    serie.fecha = pd.to_datetime(serie.fecha, yearfirst=True)
    
    return serie

### 2.2 Generar series

#### Definir fechas

In [43]:
# Definir la duración de cada periodo y calcular los periodos
step = datetime.timedelta(days=30)
periodos = generate_dates(start_date, final_date, step)
print('Nº de periodos en los que se divide el periodo de estudio: ', len(periodos))

Nº de periodos en los que se divide el periodo de estudio:  828


#### Definir las estaciones

In [44]:
# Definir las estaciones a extraer
if estaciones[0] == 'todas':
    stn_AEMET_sub = stn_AEMET
else:
    stn_AEMET_sub = stn_AEMET.loc[estaciones]
        
print('Nº de estaciones a extraer: ', stn_AEMET_sub.shape[0])

Nº de estaciones a extraer:  9


#### Extraer, organizar y exportar las series

In [45]:
# Extraer, reorganizar y exportar las series de cada una de las estaciones
for stn in stn_AEMET_sub.index[1:]:
    # Cargar la API key
    api_key = open(ruta_API).read().rstrip()
    querystring = {"api_key": api_key}
    
    # Extraer los datos
    raw_data = extract_data(stn, periodos)
    
    # Convertir los datos en numeros reales
    data = parse_data(raw_data)

    # Reorganizar la serie en forma de data frame
    serie = serie_df(data)
    
    # Exportar la serie
    serie.to_csv(ruta_series + stn + '_' + stn_AEMET_sub.loc[stn, 'nombre'] + '.csv', index=None, na_rep='NaN')
    
    del raw_data, data, serie


5270B  -  1950-01-01T00:00:00UTC  -  1950-01-31T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/90c53989', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5270B  -  1950-01-31T00:00:00UTC  -  1950-03-02T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/99e2acf1', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5270B  -  1950-03-02T00:00:00UTC  -  1950-04-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/0ebeb779', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5270B  -  1950-04-01T00:00:00UTC  -  1950-05-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/770f2985', 'metadatos': 'https://opendata.aemet.es/op

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
data_url = r.json()['datos']
r_data = requests.get(data_url, params=querystring, verify=False)

In [38]:
r_data

<Response [200]>

In [40]:
r_data.json()

{'descripcion': 'datos expirados', 'estado': 404}